In [1]:
import sys
sys.path.append('/content')
from src.fluvius import USGS_Water_DB, USGS_Station

%load_ext autoreload
%autoreload 2

db = USGS_Water_DB()
db.get_station_df()
#db.station_df.to_csv('/content/data/usgs_station_metadata.csv', index=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
There is no [linux64] chromedriver for browser 90.0.4430 in cache
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Driver has been saved in cache [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24]


In [3]:
db.station_df

,site_no,site_name,Latitude,Longitude,geometry
0,01472157,"French Creek near Phoenixville, PA",40.1515,-75.6013,POINT (-75.60130 40.15150)
1,01473169,Valley Creek at PA Turnpike Bridge near Valley...,40.0793,-75.4607,POINT (-75.46070 40.07930)
2,01478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77080 39.74750)
3,01479820,"Red Clay Creek near Kennett Square, PA",39.8168,-75.6916,POINT (-75.69160 39.81680)
4,01480300,"West Branch Brandywine Creek near Honey Brook, PA",40.0729,-75.8608,POINT (-75.86080 40.07290)
...,...,...,...,...,...
168,393806095273700,"Atchison County Lake near Horton, KS",39.6350,-95.4603,POINT (-95.46030 39.63500)
169,393817095260100,"Clear Creek at Decator Road near Horton, KS",39.6381,-95.4336,POINT (-95.43360 39.63810)
170,394126096073500,Black Vermillion River Tributary above Central...,39.6906,-96.1264,POINT (-96.12640 39.69060)
171,394146096085500,"Centralia Lake near Centralia, KS",39.6961,-96.1486,POINT (-96.14860 39.69610)


In [5]:
#water_data = USGS_Station(site_no, verbose=True, instantaneous=True)

In [ ]:
d_list = []
for site_no in db.station_df.site_no: #full loop for all stations took 24 hours...
    #instantiate water data
    water_data = USGS_Station(site_no, instantaneous=True) 
    water_data.get_water_df(write_to_csv=True)
    #we would likely append this data to the USGS_Water_DB dataframe?
    #especially if the csv is already collected, open it locally?
    d_list.append(water_data)
    #currently only 39 complete stations which is a problem...





====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


Wrote /content/data/01472157_data.csv!




====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


Wrote /content/data/01473169_data.csv!


In [8]:
import sys
sys.path.append('/content')
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import fsspec
from glob import glob

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}

fs = fsspec.filesystem('az',\
                       account_name=storage_options['account_name'],\
                       account_key=storage_options['account_key'])   

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
station_list = pd.DataFrame([{'site_no':os.path.basename(f).split('_')[0]} for f in glob('/content/data/*')])
out_metadata = station_list.merge(db.station_df, on='site_no').reset_index(drop=True)
#now open each station and write the station data to each folder
for f in glob('/content/data/*'):
    site_no = os.path.basename(f).split('_')[0]
    outfile = f'az://usgsi-data/stations/{site_no}.csv'
    c = pd.read_csv(f)
    c.to_csv(outfile, index=False, storage_options=storage_options)
out_metadata.to_csv('az://usgsi-data/usgsi_station_metadata.csv', index=False, storage_options=storage_options)

In [15]:
out_metadata

,site_no,site_name,Latitude,Longitude,geometry
0,07182250,"Cottonwood River at Plymouth, KS",38.3975,-96.3561,POINT (-96.35610 38.39750)
1,07182390,"Neosho River near Neosho Rapids, KS",38.3680,-96.0000,POINT (-96.00000 38.36800)
2,09326500,"Ferron Creek (Upper Station) near Ferron, UT",39.1041,-111.2170,POINT (-111.21700 39.10410)
3,09327000,Ferron Creek Below Millsite Res & Divs Near Fe...,39.0953,-111.1790,POINT (-111.17900 39.09530)
4,01673000,"Pamunkey River Near Hanover, VA",37.7676,-77.3322,POINT (-77.33220 37.76760)
5,06805500,"Platte River at Louisville, NE",41.0152,-96.1577,POINT (-96.15770 41.01520)
6,11455146,Liberty Cut at Little Holland Tract near Court...,38.3288,-121.6680,POINT (-121.66800 38.32880)
7,06795500,"Shell Creek near Columbus, NE",41.5261,-97.2817,POINT (-97.28170 41.52610)
8,01649190,"Paint Branch Near College Park, MD",39.0331,-76.9643,POINT (-76.96430 39.03310)
9,01478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77080 39.74750)
